# Sistema Juridico RAG Avancado - Chat Juridico Robusto

Este notebook implementa um chat juridico alinhado a arquitetura completa solicitada, cobrindo:
- Camada de apresentacao (notebook como front-end de referencia)
- Camada de aplicacao (agente juridico RAG, busca hibrida, indexacao)
- Camada de dados (PageIndex, vetorial, cache, Google Drive)
- Camada de infraestrutura (LLMs, APIs externas, logs, monitoramento)

A implementacao e um esqueleto robusto com logs, auditoria e pontos de extensao para integra-lo com Docling, PageIndex e fontes oficiais.


## Arquitetura Completa (referencia)

```
╔══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
║                                                                                                                                                          ║
║                                                SISTEMA JURÍDICO RAG AVANÇADO - ARQUITETURA COMPLETA                                                      ║
║                                                                                                                                                          ║
╠══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╣
║                                                                                                                                                          ║
║  ======================================================================================================================================================  ║
║  |                                                        CAMADA DE APRESENTAÇÃO                                                                      |  ║
║  ======================================================================================================================================================  ║
║  |                                                                                                                                                     |  ║
║  |  +--------------------------------+     +--------------------------------+     +--------------------------------+     +------------------------+   |  ║
║  |  |     Interface Web (Streamlit)  |     |   Notebooks Colab / Jupyter   |     |       API REST (FastAPI)       |     |   Aplicação Móvel      |   |  ║
║  |  |                                |     |                                |     |                                |     |                        |   |  ║
║  |  | - Consultas interativas        |     | - Experimentação e análise    |     | - Integração com sistemas      |     | - Consultas rápidas    |   |  ║
║  |  | - Visualização de resultados   |     | - Desenvolvimento de modelos  |     | - Microserviços               |     | - Notificações         |   |  ║
║  |  | - Painéis de controle          |     | - Demonstrações               |     | - Automação de fluxos         |     |                        |   |  ║
║  |  +--------------------------------+     +--------------------------------+     +--------------------------------+     +------------------------+   |  ║
║  |                                                                                                                                                     |  ║
║  ======================================================================================================================================================  ║
║                                                                                                                                                          ║
║  ======================================================================================================================================================  ║
║  |                                                        CAMADA DE APLICAÇÃO                                                                         |  ║
║  ======================================================================================================================================================  ║
║  |                                                                                                                                                     |  ║
║  |  +--------------------------------+     +--------------------------------+     +--------------------------------+     +------------------------+   |  ║
║  |  |      Agente Jurídico RAG       |     |   Sistema de Busca Híbrida    |     |   Sistema de Indexação        |     |   Sistema de Download  |   |  ║
║  |  |                                |     |                                |     |                                |     |                        |   |  ║
║  |  | - Análise de consultas         |     | - Busca em múltiplos índices  |     | - PageIndex (árvore)          |     | - Fontes oficiais      |   |  ║
║  |  | - Síntese de contexto          |     | - Combina PageIndex + Vetorial|     | - Índices vetoriais           |     | - Jurisprudência STF   |   |  ║
║  |  | - Geração de respostas         |     | - Ranqueamento por relevância |     | - Índices híbridos            |     | - Leis federais        |   |  ║
║  |  | - Verificação de fatos         |     | - Filtros por fonte/tipo      |     | - Metadados dos documentos    |     | - Processos/modelos    |   |  ║
║  |  +--------------------------------+     +--------------------------------+     +--------------------------------+     +------------------------+   |  ║
║  |                                                                                                                                                     |  ║
║  ======================================================================================================================================================  ║
║                                                                                                                                                          ║
║  ======================================================================================================================================================  ║
║  |                                                        CAMADA DE DADOS                                                                             |  ║
║  ======================================================================================================================================================  ║
║  |                                                                                                                                                     |  ║
║  |  +--------------------------------+     +--------------------------------+     +--------------------------------+     +------------------------+   |  ║
║  |  |     Armazenamento Vetorial     |     |    PageIndex (Árvores)        |     |   Cache Inteligente           |     |   Google Drive         |   |  ║
║  |  |                                |     |                                |     |                                |     |                        |   |  ║
║  |  | - Embeddings de documentos     |     | - Estruturas hierárquicas     |     | - Consultas frequentes        |     | - Documentos brutos     |   |  ║
║  |  | - Similaridade semântica       |     | - Raciocínio sobre estrutura  |     | - Resultados de busca         |     | - Índices serializados  |   |  ║
║  |  | - Modelos locais/OpenAI        |     | - Navegação contextual        |     | - Embeddings                  |     | - Logs e auditoria      |   |  ║
║  |  +--------------------------------+     +--------------------------------+     +--------------------------------+     +------------------------+   |  ║
║  |                                                                                                                                                     |  ║
║  ======================================================================================================================================================  ║
║                                                                                                                                                          ║
║  ======================================================================================================================================================  ║
║  |                                                     CAMADA DE INFRAESTRUTURA                                                                       |  ║
║  ======================================================================================================================================================  ║
║  |                                                                                                                                                     |  ║
║  |  +--------------------------------+     +--------------------------------+     +--------------------------------+     +------------------------+   |  ║
║  |  |       APIs Externas            |     |        Modelos LLM            |     |     Sistema de Logs          |     |   Monitoramento        |   |  ║
║  |  |                                |     |                                |     |                                |     |                        |   |  ║
║  |  | - PageIndex API                |     | - OpenAI GPT-4/3.5            |     | - Logs estruturados           |     | - Métricas de uso      |   |  ║
║  |  | - STF APIs (quando disponível) |     | - Modelos locais (opcional)   |     | - Auditoria de consultas      |     | - Desempenho do sistema|   |  ║
║  |  | - Fontes oficiais de leis      |     | - Fine-tuning específico      |     | - Rastreamento de erros       |     | - Alertas              |   |  ║
║  |  +--------------------------------+     +--------------------------------+     +--------------------------------+     +------------------------+   |  ║
║  |                                                                                                                                                     |  ║
║  ======================================================================================================================================================  ║
║                                                                                                                                                          ║
║  ======================================================================================================================================================  ║
║  |                                                     FLUXO DE PROCESSAMENTO                                                                         |  ║
║  ======================================================================================================================================================  ║
║  |                                                                                                                                                     |  ║
║  |  1. Download e Indexação                                                                                                                           |  ║
║  |     [Fontes] → [Sistema Download] → [Documentos brutos] → [Sistema Indexação] → [Índices PageIndex/Vetoriais] → [Armazenamento]                   |  ║
║  |                                                                                                                                                     |  ║
║  |  2. Processamento de Consulta                                                                                                                      |  ║
║  |     [Consulta] → [Agente RAG] → [Análise] → [Sistema Busca Híbrida] → [Recuperação] → [Síntese] → [Geração] → [Verificação] → [Resposta]         |  ║
║  |                                                                                                                                                     |  ║
║  |  3. Armazenamento e Auditoria                                                                                                                      |  ║
║  |     [Resposta] → [Cache] → [Logs] → [Google Drive] → [Relatórios]                                                                                 |  ║
║  |                                                                                                                                                     |  ║
║  ======================================================================================================================================================  ║
║                                                                                                                                                          ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╝
```


## Dependencias e configuracao

Ajuste chaves e caminhos conforme seu ambiente (local/Colab).

In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Any, Dict, List, Optional
import hashlib
import json

@dataclass(frozen=True)
class ConfigSistema:
    pageindex_api_url: str = "https://api.pageindex.ai/v1/index"
    pageindex_api_key: str = "SUA_CHAVE_AQUI"
    chatindex_dir: Path = Path("./chatindex")
    drive_root: Path = Path("./Juridico_Unificado")
    audit_dir: Path = Path("./Juridico_Unificado/05_Auditoria")
    cache_dir: Path = Path("./cache_juridico")
    vetor_dir: Path = Path("./vetor_index")

config = ConfigSistema()
config

## Camada de infraestrutura: auditoria e logs

Logs encadeados por hash para rastreabilidade e integridade.

In [ ]:
class SistemaAuditoriaUnificado:
    def __init__(self, audit_dir: Path):
        self.audit_dir = audit_dir
        self.audit_dir.mkdir(parents=True, exist_ok=True)
        self.log_central: List[Dict[str, Any]] = []
        self.hash_registry: Dict[str, Dict[str, str]] = {}

    def registrar_evento(self, categoria: str, evento: Dict[str, Any]) -> str:
        evento_id = f"evt_{hashlib.md5(str(evento).encode()).hexdigest()[:10]}"
        evento["timestamp"] = evento.get("timestamp", datetime.now().isoformat())

        if self.log_central:
            evento["hash_anterior"] = self.hash_registry[self.log_central[-1]["evento_id"]]["hash"]

        hash_atual = hashlib.md5(json.dumps(evento, sort_keys=True).encode()).hexdigest()
        self.hash_registry[evento_id] = {"hash": hash_atual, "timestamp": evento["timestamp"]}

        registro = {**evento, "evento_id": evento_id, "hash_atual": hash_atual}
        self.log_central.append(registro)
        self._persistir_log(categoria, registro)
        return evento_id

    def _persistir_log(self, categoria: str, registro: Dict[str, Any]) -> None:
        destino = self.audit_dir / f"{categoria}.jsonl"
        with destino.open("a", encoding="utf-8") as handle:
            handle.write(json.dumps(registro, ensure_ascii=False) + "\n")

    def resumo_eventos(self) -> List[Dict[str, Any]]:
        return self.log_central


## Camada de dados: caches e indices (stubs)

Estruturas simples para demonstrar o fluxo hibrido PageIndex + vetorial.

In [ ]:
class CacheInteligente:
    def __init__(self, cache_dir: Path):
        self.cache_dir = cache_dir
        self.cache_dir.mkdir(parents=True, exist_ok=True)
        self.memoria: Dict[str, Any] = {}

    def get(self, chave: str) -> Optional[Any]:
        return self.memoria.get(chave)

    def set(self, chave: str, valor: Any) -> None:
        self.memoria[chave] = valor


class PageIndexStore:
    def __init__(self):
        self.arvores: Dict[str, Dict[str, Any]] = {}

    def salvar(self, documento_id: str, arvore: Dict[str, Any]) -> None:
        self.arvores[documento_id] = arvore

    def buscar(self, consulta: str) -> List[Dict[str, Any]]:
        return [{"documento_id": doc_id, "match": "Resumo da arvore"} for doc_id in self.arvores]


class VetorStore:
    def __init__(self, vetor_dir: Path):
        self.vetor_dir = vetor_dir
        self.vetor_dir.mkdir(parents=True, exist_ok=True)
        self.embeddings: Dict[str, List[float]] = {}

    def indexar(self, documento_id: str, embedding: List[float]) -> None:
        self.embeddings[documento_id] = embedding

    def buscar(self, consulta: str) -> List[Dict[str, Any]]:
        return [{"documento_id": doc_id, "score": 0.5} for doc_id in self.embeddings]


## Camada de aplicacao: download, indexacao e busca hibrida

Os componentes abaixo sao conectados pelo orquestrador e usados pelo chat juridico.

In [ ]:
class SistemaDownload:
    def baixar(self, fonte: str, consulta: str) -> Dict[str, Any]:
        return {"fonte": fonte, "documento_id": f"doc_{fonte.lower()}", "conteudo": f"Conteudo de {fonte}"}


class SistemaIndexacao:
    def __init__(self, pageindex_store: PageIndexStore, vetor_store: VetorStore):
        self.pageindex_store = pageindex_store
        self.vetor_store = vetor_store

    def indexar_documento(self, documento: Dict[str, Any]) -> Dict[str, Any]:
        arvore = {
            "documento_id": documento["documento_id"],
            "estrutura_arvore": {"raiz": {"titulo": documento["documento_id"]}},
        }
        self.pageindex_store.salvar(documento["documento_id"], arvore)
        self.vetor_store.indexar(documento["documento_id"], [0.1, 0.2, 0.3])
        return arvore


class SistemaBuscaHibrida:
    def __init__(self, pageindex_store: PageIndexStore, vetor_store: VetorStore):
        self.pageindex_store = pageindex_store
        self.vetor_store = vetor_store

    def buscar(self, consulta: str) -> Dict[str, Any]:
        resultados_pageindex = self.pageindex_store.buscar(consulta)
        resultados_vetoriais = self.vetor_store.buscar(consulta)
        return {
            "pageindex": resultados_pageindex,
            "vetorial": resultados_vetoriais,
        }


## Camada de aplicacao: agente juridico RAG

O agente consolida resultados, gera resposta e registra auditoria.

In [ ]:
class AgenteJuridicoRAG:
    def __init__(self, busca: SistemaBuscaHibrida, auditoria: SistemaAuditoriaUnificado, cache: CacheInteligente):
        self.busca = busca
        self.auditoria = auditoria
        self.cache = cache

    def responder(self, consulta: str) -> Dict[str, Any]:
        cache_hit = self.cache.get(consulta)
        if cache_hit:
            return {**cache_hit, "cache": True}

        resultados = self.busca.buscar(consulta)
        resposta = {
            "consulta": consulta,
            "resposta": f"Resposta juridica simulada para: {consulta}",
            "fontes": resultados,
        }
        self.cache.set(consulta, resposta)
        self.auditoria.registrar_evento("consultas", {
            "tipo": "resposta",
            "consulta": consulta,
            "hash_resposta": hashlib.md5(json.dumps(resposta, ensure_ascii=False).encode()).hexdigest(),
        })
        return resposta


## Orquestrador e Chat Juridico

Integra todos os componentes: download, indexacao, busca hibrida e auditoria.

In [ ]:
class SistemaJuridicoUnificado:
    def __init__(self, config: ConfigSistema):
        self.auditoria = SistemaAuditoriaUnificado(config.audit_dir)
        self.cache = CacheInteligente(config.cache_dir)
        self.pageindex_store = PageIndexStore()
        self.vetor_store = VetorStore(config.vetor_dir)

        self.download = SistemaDownload()
        self.indexacao = SistemaIndexacao(self.pageindex_store, self.vetor_store)
        self.busca = SistemaBuscaHibrida(self.pageindex_store, self.vetor_store)
        self.agente = AgenteJuridicoRAG(self.busca, self.auditoria, self.cache)

    def preparar_fontes(self, consulta: str, fontes: List[str]) -> None:
        for fonte in fontes:
            documento = self.download.baixar(fonte, consulta)
            self.indexacao.indexar_documento(documento)
            self.auditoria.registrar_evento("indexacao", {
                "tipo": "documento_indexado",
                "fonte": fonte,
                "documento_id": documento["documento_id"],
            })

    def responder(self, consulta: str) -> Dict[str, Any]:
        self.auditoria.registrar_evento("consultas", {
            "tipo": "consulta",
            "consulta": consulta,
        })
        return self.agente.responder(consulta)


class ChatJuridico:
    def __init__(self, sistema: SistemaJuridicoUnificado):
        self.sistema = sistema
        self.historico: List[Dict[str, Any]] = []

    def enviar(self, consulta: str, fontes: List[str]) -> Dict[str, Any]:
        self.sistema.preparar_fontes(consulta, fontes)
        resposta = self.sistema.responder(consulta)
        self.historico.append({"consulta": consulta, "resposta": resposta})
        return resposta


## Exemplo de uso do Chat Juridico

Este exemplo simula consulta com fontes oficiais (STF, STJ, Planalto).

In [ ]:
sistema = SistemaJuridicoUnificado(config)
chat = ChatJuridico(sistema)

chat.enviar("Quais sao os requisitos para tutela de urgencia?", ["STF", "STJ", "Planalto"])

## Proximos passos

1. Conectar Docling para extracao estruturada.
2. Integrar PageIndex real via API ou MCP.
3. Adicionar vetores com embeddings reais (OpenAI ou local).
4. Implementar scraping resiliente (Playwright + cache).
5. Persistir historico no ChatIndex e Google Drive.
